In [17]:
import numpy as np
import pandas as pd

df = pd.read_csv('/Users/noahv/Data-Mining-techniques/course_dmt/ass2/datasets/training_set_VU_DM.csv')

I use this function to check if there is a change in values over time for a column of a prop_id


In [3]:
def change_in_values_over_time(df, col, change = True):
    has_change_over_time = df.groupby('prop_id')[col].apply(lambda x: x.diff().any())
    prop_ids_change = has_change_over_time[has_change_over_time == change].index
    print(len(prop_ids_change))
    #print(prop_ids_change)

#change_in_values_over_time(df, 'prop_location_score2')
#change_in_values_over_time(df, 'prop_review_score')
change_in_values_over_time(df, 'prop_starrating')


0


This function can be used to see if there is a change from a NaN value to a non NaN value

In [12]:
def change_in_values_over_time(df, col, change = False):
    has_change_over_time = df.groupby('prop_id')[col].apply(lambda x: x.diff().notna().any())
    prop_ids_change = has_change_over_time[has_change_over_time == change].index
    print(len(prop_ids_change))
    #print(prop_ids_change)

change_in_values_over_time(df, 'prop_location_score2')



42898
Int64Index([     1,      5,      7,     10,     12,     20,     21,     22,
                25,     27,
            ...
            140783, 140784, 140785, 140789, 140793, 140794, 140796, 140802,
            140806, 140820],
           dtype='int64', name='prop_id', length=42898)


Calculate the price per day for each hotel

In [13]:

def get_price_per_night(df : pd.DataFrame) -> pd.DataFrame:  

    unique_properties = df['prop_id'].unique()
    new_df = pd.DataFrame()

    count = 0
    for prop_i in unique_properties:
        df_for_prop_i = df[df['prop_id'] == prop_i]

        if (df_for_prop_i['srch_length_of_stay'] == 1).any():
            instances_with_length_of_stay_1 = df_for_prop_i[df_for_prop_i['srch_length_of_stay'] == 1]
            mean_price_per_night = instances_with_length_of_stay_1['price_usd'].mean()
            df_for_prop_i['mean_price_per_night'] = mean_price_per_night
            new_df = new_df.append(df_for_prop_i)

        else:
            df_for_prop_i['mean_price_per_night'] = np.nan
            print(df_for_prop_i['prop_id'])
            count += 1
            print(count)
            new_df = new_df.append(df_for_prop_i)

    return new_df


df = get_price_per_night(df)



C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


519        2169
189298     2169
240601     2169
383880     2169
464645     2169
533732     2169
674561     2169
886337     2169
1226165    2169
1280291    2169
1382780    2169
1407544    2169
1440433    2169
1618671    2169
1855488    2169
1918107    2169
1977394    2169
2579080    2169
2619185    2169
2662136    2169
3101368    2169
3206201    2169
3276015    2169
3289144    2169
3844833    2169
3916977    2169
4187180    2169
4385248    2169
4442748    2169
4500168    2169
4571954    2169
4604906    2169
4613500    2169
4671031    2169
4760591    2169
4769211    2169
4828603    2169
4901191    2169
Name: prop_id, dtype: int64
1
526        33931
189306     33931
240608     33931
280901     33931
383888     33931
464652     33931
533736     33931
617234     33931
886341     33931
1226172    33931
1382787    33931
1618678    33931
1633136    33931
1977401    33931
2579087    33931
2619192    33931
2721240    33931
3101376    33931
3198384    33931
3206206    33931
3276021    33931
32891

KeyboardInterrupt: 

In [9]:
# This function is way faster, not sure if it works properly yet though
def get_price_per_night(df: pd.DataFrame) -> pd.DataFrame:
    mask = df['srch_length_of_stay'] == 1
    df['mean_price_per_night'] = np.nan
    df.loc[mask, 'mean_price_per_night'] = df.loc[mask].groupby('prop_id')['price_usd'].transform('mean')
    return df

new_df = get_price_per_night(df)

In [14]:
new_df = new_df[['prop_id', 'price_usd', 'srch_length_of_stay', 'mean_price_per_night']]
new_df.head(0)

,prop_id,price_usd,srch_length_of_stay,mean_price_per_night


This code is used to impute the 'prop_loc_score2' column values

In [4]:
nan_instances = df[df['prop_location_score2'].isnull()]
print_df = nan_instances[['prop_id', 'prop_location_score2']]
print_df
print(len(print_df['prop_id'].unique()))

1090348
106909


In [14]:
# Function to impute the prop_location_score2 values that are NaN based on the prop_location_score2 values of the same prop_id
def impute_prop_loc_score2_using_future_score2(df : pd.DataFrame) -> pd.DataFrame:

    # Retrieve the prop_ids that have a change in prop_location_score2 from NaN to a float over time
    has_change_over_time = df.groupby('prop_id')['prop_location_score2'].apply(lambda x: x.diff().notna().any())
    prop_ids_change = has_change_over_time[has_change_over_time == True].index

    # For each prop_id that has a change in prop_location_score2 from NaN to a float over time, impute the NaN values with the first non-NaN value
    for prop_id in prop_ids_change:
        mask = (df['prop_id'] == prop_id) & (df['prop_location_score2'].isnull())
        non_nan_value = df.loc[(df['prop_id'] == prop_id) & (df['prop_location_score2'].notna()), 'prop_location_score2'].values[0]
        df.loc[mask, 'prop_location_score2'] = non_nan_value

    return df

df = impute_prop_loc_score2_using_future_score2(df)
df

KeyboardInterrupt: 

In [7]:
# Impute remaning NaN values using the mean prop_locatioin_score2 values for each prop_location_score1 value
def impute_prop_location_score2_using_mean_score1(df : pd.DataFrame) -> pd.DataFrame:

    # Create an extra column with rounded 'prop_location_score1'
    df['rounded_score1'] = df['prop_location_score1'].round()

    # Calculate the mean of 'prop_location_score2' for each rounded integer value
    mean_score2_by_rounded = df.groupby('rounded_score1')['prop_location_score2'].mean()

    # Impute missing values of 'prop_location_score2' based on rounded integer values
    df['prop_location_score2'].fillna(df['rounded_score1'].map(mean_score2_by_rounded), inplace=True)

    # Drop the extra column 'rounded_score1' if no longer needed
    df.drop('rounded_score1', axis=1, inplace=True)

    return df

dummy_df = impute_prop_location_score2_using_mean_score1(df)
dummy_df['prop_location_score2'].isnull().sum()

0

IMPUTE PROP_STARRATING <br>
This part is used to impute the prop_starrating

In [ ]:
def impute_prop_starrating(df : pd.DataFrame) -> pd.DataFrame:


    return df

IMPUTE PROP_REVIEW SCORE    <br>
This part is used to impute the prop_review score values

In [6]:
# To check if there are instances in the dataframe where two specific columns contain NaN values in the same row
dummy_df = df[(df['prop_starrating'] == 0) & (df['prop_review_score'].isna())]
print(len(dummy_df))


0


In [9]:
def impute_prop_review_score(df : pd.DataFrame) -> pd.DataFrame:

    # Calculate the mean of 'prop_location_score2' for each rounded integer value
    mean_review_score_for_starrating = df.groupby('prop_starrating')['prop_review_score'].mean()

    # Impute missing values of 'prop_location_score2' based on rounded integer values
    df['prop_review_score'].fillna(df['prop_starrating'].map(mean_review_score_for_starrating), inplace=True)

    return df

dummy_df = impute_prop_review_score(df)
dummy_df['prop_review_score'].isnull().sum()
    

0

In [11]:
dummy_df = dummy_df[['prop_review_score', 'prop_starrating']]
#dummy_df.head(20)

IMPUTE ORIG_DESTINATION_DISTANCE


In [18]:
df['orig_destination_distance'].isna().sum()

1607782

In [19]:
# Calculate the mean distance per hotel and impute the NaN values with the mean distance
def impute_orig_destination_distance(df : pd.DataFrame) -> pd.DataFrame:
    
    # Calculate the mean distance per hotel
    mean_distance_per_hotel = df.groupby('prop_id')['orig_destination_distance'].mean()
    
    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['prop_id'].map(mean_distance_per_hotel), inplace=True)

    # BUT there are also hotels that contain only NaN values for orig_destination_distance, 
    # for these we impute it with the mean distance for the country_id
    # Use visitor_location_country_id or country_id?

    # Calculate the mean distance per country
    mean_distance_per_country = df.groupby('visitor_location_country_id')['orig_destination_distance'].mean()

    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['visitor_location_country_id'].map(mean_distance_per_country), inplace=True)
    
    return df

dummy_df = impute_orig_destination_distance(df)
dummy_df['orig_destination_distance'].isna().sum()

0

OUTLIER REMOVAL <br>
We can change some values and experiment with how our model performs. I provided some research for this outlier in EDA_noah.ipynb regarding the min - max values, boxplots and the number of values that exceed a particular threshold. <br>

I also read that in recommender systems outlier removal might not even be necessary because it could be due to the fact of extreme behaviour of some users.

In [8]:
def remove_outliers(df: pd.DataFrame):
    # remove outliers
    df = df[df['price_usd'] < 600000]
    df = df[df['srch_length_of_stay'] < 45]
    df = df[df['srch_booking_window'] < 500]
    # df = df[df['gross_bookings_usd'] < 45000]
    # df = df[df['srch_query_affinity_score'] > -300]
    return df

new_df = remove_outliers(df)
new_df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
12,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2,3.0,...,NaN,NaN,NaN,NaN,0.0,0.0,11.0,1,114.29,1
63,6,2013-06-05 12:27:51,14,100,NaN,NaN,100,104251,3,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,162.38,1
68,8,2013-03-20 17:50:44,5,219,NaN,NaN,219,27669,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1,96.41,1
194,21,2013-02-18 17:16:25,15,55,NaN,NaN,216,61662,3,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,222.58,1
211,25,2012-12-11 13:03:42,5,219,NaN,NaN,219,23228,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1,47.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958241,332776,2012-11-17 10:02:22,5,32,5.0,321.49,55,106758,4,4.0,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1,428.84,1
4958275,332781,2013-03-03 15:12:59,5,219,NaN,NaN,219,18007,3,3.5,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1,129.94,1
4958304,332782,2012-11-08 18:24:21,19,158,NaN,NaN,158,68398,5,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,288.25,1
4958321,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,34966,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,301.40,1


Final Pipeline

In [ ]:
def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:

def add_nan_column(df: pd.DataFrame, cols):
    for col in cols:
        df[col + '_is_nan'] = df[col].isna()
    return df

def add_zero_column(df: pd.Dataframe, cols):
    for col in cols:
        df[col + '_is_zero'] = df[col] == 0
    return df

def calc_price_per_night(df: pd.DataFrame):

#def impute_prop_loc_score2(df: pd.DataFrame):

#def impute_prop_review_score(df: pd.DataFrame):

#def impute_prop_starrating(df: pd.DataFrame):

def impute_query_affinity_score(df: pd.DataFrame):

def impute_orig_destination_distance(df: pd.DataFrame):

def define_target(df: pd.DataFrame):

def one_hot_encode(df: pd.DataFrame, cols):
    for col in cols:
        df = pd.get_dummies(df, columns=[col], prefix=col)
    return df

def delete_id_columns(df: pd.DataFrame):
    for col in df.columns:
        if col.endswith('_id'):
            df = df.drop(col, axis=1)
    return df